# Project 4: West Nile Virus - Modelling

---
# Table of Contents

1. Import Libraries and Data
2. Preprocessing
3. Modelling
4. Evaluating and Interpreting Results
5. Generating Predictions for Kaggle
6. Conclusion
---


# Import Libraries and Data

In [28]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer, make_column_selector

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, roc_auc_score, RocCurveDisplay

In [2]:
# import training dataset
train = pd.read_csv('../output_data/merged_train.csv')

# data cleaning
train.set_index('date', inplace=True)
train.drop(columns='nummosquitos', inplace=True)

# check the df columns
print(f'Features in training set : {train.columns.tolist()}')

Features in training set : ['species', 'latitude', 'longitude', 'wnvpresent', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'preciptotal', 'temp_fluct', 'daylight', 'relative_hum']


In [3]:
# define feature matrix and target vector
X = train.drop(columns='wnvpresent')
y = train['wnvpresent']

# Preprocessing 
## Train-validation  split

In [4]:
# split dataset into train and validation set
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y,
                                                  random_state=42,
                                                  stratify=y)

In [5]:
# check the shape of training and testing vectors
print(" Train Set Shape ".center(27, '='))
print(f'Features X_train: {X_train.shape}')
print(f'Targets y_train:  {y_train.shape}')
print()
print(" Validation Set Shape ".center(26, '='))
print(f'Features X_val:   {X_val.shape}')
print(f'Targets y_val:    {y_val.shape}')

===== Train Set Shape =====
Features X_train: (7879, 15)
Targets y_train:  (7879,)

== Validation Set Shape ==
Features X_val:   (2627, 15)
Targets y_val:    (2627,)


## Features transformation 

In [6]:
# instantiate transformers 
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
sc = StandardScaler()


# instantiate transformers into column transformer 
ct = make_column_transformer(
    (sc, make_column_selector(dtype_include='number')),
    (ohe, make_column_selector(dtype_exclude='number')),
    remainder='passthrough', 
    verbose_feature_names_out=False
)

# transform feature matrix
X_train = pd.DataFrame(ct.fit_transform(X_train), 
                       columns=ct.get_feature_names_out())

X_val = pd.DataFrame(ct.transform(X_val), 
                       columns=ct.get_feature_names_out())

# check X matrix after transformation
X_train.head()

,latitude,longitude,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,temp_fluct,daylight,relative_hum,species_ERRATICUS,species_PIPIENS,species_PIPIENS/RESTUANS,species_RESTUANS,species_SALINARIUS,species_TARSALIS,species_TERRITANS
0,-0.863790,-0.329696,0.435022,-0.552766,-0.002088,0.350149,-0.412475,-0.198591,-0.328471,-0.157190,...,1.483017,-1.008347,-0.811214,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.515998,-0.176909,0.495685,0.048510,0.266496,1.120683,-0.092320,0.025375,-0.328471,0.193288,...,0.714862,-1.262680,-0.690378,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.661906,1.428458,-2.962122,-2.356596,-2.822215,-1.653240,-2.333406,-2.662212,4.515917,-1.559102,...,-1.301544,-2.105158,0.402667,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-1.013760,1.193503,0.980991,0.783404,0.937954,0.966576,1.380393,1.219858,-0.328471,1.069483,...,0.426804,-0.372515,1.018749,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.188928,0.531668,0.374358,0.716595,0.535079,0.041935,1.124269,0.921237,-0.328471,0.543766,...,-0.437370,0.819670,1.269894,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# check the df cloumns
print(f'Features in X matrix: {X_train.columns.tolist()}')

Features in X matrix: ['latitude', 'longitude', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'preciptotal', 'temp_fluct', 'daylight', 'relative_hum', 'species_ERRATICUS', 'species_PIPIENS', 'species_PIPIENS/RESTUANS', 'species_RESTUANS', 'species_SALINARIUS', 'species_TARSALIS', 'species_TERRITANS']


In [8]:
# check the shape of training and testing vectors
print(" Train Set Shape ".center(27, '='))
print(f'Features X_train: {X_train.shape}')
print(f'Targets y_train:  {y_train.shape}')
print()
print(" Validation Set Shape ".center(26, '='))
print(f'Features X_val:   {X_val.shape}')
print(f'Targets y_val:    {y_val.shape}')

===== Train Set Shape =====
Features X_train: (7879, 21)
Targets y_train:  (7879,)

== Validation Set Shape ==
Features X_val:   (2627, 21)
Targets y_val:    (2627,)


# Modelling
## Functions used

In [9]:
def fit_gs(clf, params):
    """fits a GridSearchCV to a classifier, prints best params and returns model"""
    
    gs = GridSearchCV(clf, params, cv=5, n_jobs=-1, scoring= "roc_auc")
    gs.fit(X_train, y_train)
    
    print(f"Best Parameters: {gs.best_params_}")
    
    return gs

In [10]:
def eval_model(model):
    """returns dataframe of evaluation metrics for classifier"""
    
    # get predictions
    y_pred = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:,1]
        
    # metrics for evaluating classifier
    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    
    accuracy = (tn + fn) / (tn + fp + fn + tp)
    
    if (tn + fp) != 0:
        spec = tn / (tn + fp)
    else:
        spec = 'NA'
        
    if (tp + fn) != 0:
        recall = tp / (tp + fn)
    else:
        recall = np.nan

    if (tp + fp) != 0:
        precision = tp / (tp + fp)
    else:
        precision = np.nan
        
    if recall == np.nan or precision == np.nan:
        f1 = np.nan
    else:
        f1 = 2*((precision*recall)/(precision+recall))

    roc = roc_auc_score(y_val, y_proba)
    
    try:
        model_name = str(model.estimator)[:-2]
        if len(model_name) > 30:
            model_name = model_name[:13]
    except:
        model_name = 'Dummy'
        
    df = pd.DataFrame([np.round([accuracy, spec, recall, precision, f1, fp, fn, roc], 3)], 
                      columns = ["Accuracy", 
                                 "Specificity", 
                                 "Recall", 
                                 "Precision", 
                                 "F1", 
                                 "False Positives", 
                                 "False Negatives", 
                                 "ROC-AUC"], 
                      index = [model_name],
                      dtype='str')
    return df

## (0) Dummy model

In [13]:
# fit dummy model
dummy = DummyClassifier(strategy="most_frequent", random_state=42)
dummy.fit(X_train, y_train)

dummy_result = eval_model(dummy)
dummy_result

,Accuracy,Specificity,Recall,Precision,F1,False Positives,False Negatives,ROC-AUC
Dummy,1.0,1.0,0.0,NaN,NaN,0.0,138.0,0.5


## (1) Random forest classifier

In [14]:
rfc_params = {"n_estimators": [10, 50, 100, 250], 
              "max_depth": [5, 10, 20],
              "class_weight": [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}],
              "random_state": [42]
             }
rfc = fit_gs(RandomForestClassifier(), rfc_params)

Best Parameters: {'class_weight': {0: 1, 1: 10}, 'max_depth': 10, 'n_estimators': 100, 'random_state': 42}


In [15]:
rfc_result = eval_model(rfc)
rfc_result

,Accuracy,Specificity,Recall,Precision,F1,False Positives,False Negatives,ROC-AUC
RandomForestClassifier,0.884,0.903,0.449,0.204,0.281,242.0,76.0,0.824


## (2) Support vector classifier

In [16]:
svc_params = {"C": [10**-2, 10**-1, 10**0, 10**1, 10**2],
              "class_weight": [None, {0:1,1:10}, {0:1,1:25}],
              "probability": [True],
              "random_state": [42]
             }
svc = fit_gs(SVC(), svc_params)

Best Parameters: {'C': 1, 'class_weight': {0: 1, 1: 10}, 'probability': True, 'random_state': 42}


In [17]:
svc_result = eval_model(svc)
svc_result

,Accuracy,Specificity,Recall,Precision,F1,False Positives,False Negatives,ROC-AUC
SVC,0.808,0.832,0.616,0.169,0.265,419.0,53.0,0.812


## (3) Logistic regression

In [19]:
lr_params = {"C": [10**-2, 10**-1, 10**0, 10**1, 10**2],
             "penalty": ['l1', 'l2'],
             "max_iter": [20, 50, 100, 200],
             "class_weight": [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}],
             "solver": ['liblinear'],
             "random_state": [42]
            }

lr = fit_gs(LogisticRegression(), lr_params)

Best Parameters: {'C': 1, 'class_weight': {0: 1, 1: 5}, 'max_iter': 50, 'penalty': 'l1', 'random_state': 42, 'solver': 'liblinear'}


In [20]:
lr_result = eval_model(lr)
lr_result

,Accuracy,Specificity,Recall,Precision,F1,False Positives,False Negatives,ROC-AUC
LogisticRegression,0.953,0.961,0.188,0.211,0.199,97.0,112.0,0.792


## (4) K-nearest neighbour

In [21]:
knn_params = {"n_neighbors": [5,35,65,95]}

knn = fit_gs(KNeighborsClassifier(), knn_params)

Best Parameters: {'n_neighbors': 65}


In [22]:
knn_result = eval_model(knn)
knn_result

,Accuracy,Specificity,Recall,Precision,F1,False Positives,False Negatives,ROC-AUC
KNeighborsClassifier,1.0,1.0,0.0,NaN,NaN,0.0,138.0,0.781


## (5) Gradient boosting classifier

In [23]:
gbc_params = {"n_estimators": [150, 200, 250],
              "max_depth": [3, 5],
              "random_state": [42]
             }
gbc = fit_gs(GradientBoostingClassifier(), gbc_params)

Best Parameters: {'max_depth': 3, 'n_estimators': 200, 'random_state': 42}


In [24]:
gbc_result = eval_model(gbc)
gbc_result

,Accuracy,Specificity,Recall,Precision,F1,False Positives,False Negatives,ROC-AUC
GradientBoostingClassifier,0.994,0.997,0.065,0.562,0.117,7.0,129.0,0.833


## (6) XGB classifier

In [ ]:
xgb_params = {"use_label_encoder": [False],
              "objective": ["binary:logistic"],
              "eval_metric": ['auc'],
              "learning_rate":[0.1, 0.2, 0.5],
              "max_depth": [5,8],
              "max_leaf_nodes": [None,], #tune params
              "gamma": np.linspace(5, 100, 8),
              "reg_alpha": np.linspace(0.0001, 1, 8),
              "reg_lambda": np.linspace(5, 100, 5)} #tune params

xgb = fit_gs(XGBClassifier(random_state = 42), xgb_params)

In [ ]:
xgb_result = eval_model(xgb)
xgb_result

## (7) Extra trees classifier

In [ ]:
et_params = {"max_depth": [3,5,10,20],
             "min_samples_leaf": [1, 3, 10, 20],
             "max_leaf_nodes": [None, 5, 10],
             "random_state": [42]
            }
et = fit_gs(ExtraTreesClassifier(), et_params)

In [ ]:
et_result = eval_model(et)
et_result

# Evaluating and Interpreting Results
## Summary of results for all models

In [ ]:
summary = pd.concat([dummy_result, rfc_result, svc_result, lr_result, knn_result, gbc_result, xgb_result, 
                    et_result]) 
summary

## ROC plot

In [ ]:
models = [dummy, rfc, svc, lr, knn, gbc, xgb, et]

fig, ax = plt.subplots(figsize=(10,8))
plt.plot([0, 1], [0, 1], "k--")
for i, model in enumerate(models):
    RocCurveDisplay.from_estimator(model,
                                   X_val,
                                   y_val,
                                   ax=ax,
                                   name=f'{summary.index[i]}')

## Important features

In [30]:
def get_impt_features(models):
    result = pd.DataFrame()
    for i, model in enumerate(models):
        try:
            importances = model.best_estimator_.feature_importances_
            df = pd.DataFrame(zip(X_train.columns,importances), 
                              columns=[f'{summary.index[i]}','importance']).\
                sort_values('importance', ascending=False).reset_index(drop=True)
            result = pd.concat([result,df], axis = 1)
            
        except:
            pass
        
        try:
            coef = model.best_estimator_.coef_
            df = pd.DataFrame([X_train.columns,(*coef)]).T. \
                rename(columns={0:f'{summary.index[i]}', 1:'coefficient'}). \
                sort_values('coefficient', ascending=False, key=abs).reset_index(drop=True)
            result = pd.concat([result,df], axis = 1)

        except:
            pass
        
    return result

In [ ]:
get_impt_features(models)

# Generating Predictions for Kaggle

In [32]:
# import test dataset and final weather data
test = pd.read_csv('../input_data/test.csv')
weather = pd.read_csv('../output_data/weather_final.csv')

# check test dataframe
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Id                      116293 non-null  int64  
 1   Date                    116293 non-null  object 
 2   Address                 116293 non-null  object 
 3   Species                 116293 non-null  object 
 4   Block                   116293 non-null  int64  
 5   Street                  116293 non-null  object 
 6   Trap                    116293 non-null  object 
 7   AddressNumberAndStreet  116293 non-null  object 
 8   Latitude                116293 non-null  float64
 9   Longitude               116293 non-null  float64
 10  AddressAccuracy         116293 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 9.8+ MB


In [33]:
# drop unnecessary columns
test.drop(columns=['Address', 'Block', 'Street', 'Trap', 
                   'AddressNumberAndStreet', 'AddressAccuracy'], inplace=True)

# rename columns to lowercase
test.columns = test.columns.str.lower()

# rename Species to drop 'CULEX'
test['species'] = [i[6:] for i in test['species']]

# check dataframe
test.head()

,id,date,species,latitude,longitude
0,1,2008-06-11,PIPIENS/RESTUANS,41.95469,-87.800991
1,2,2008-06-11,RESTUANS,41.95469,-87.800991
2,3,2008-06-11,PIPIENS,41.95469,-87.800991
3,4,2008-06-11,SALINARIUS,41.95469,-87.800991
4,5,2008-06-11,TERRITANS,41.95469,-87.800991


In [34]:
# check weather dataframe
weather.head()

,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,preciptotal,temp_fluct,daylight,relative_hum
0,2007-05-01,83.5,51.0,67.50,14.0,51.0,56.5,0.0,2.5,0.0,32.5,14.016667,0.554538
1,2007-05-02,59.5,42.5,51.50,-3.0,42.0,47.0,13.5,0.0,0.0,17.0,14.050000,0.698858
2,2007-05-03,66.5,47.0,57.00,2.0,40.0,49.0,8.0,0.0,0.0,19.5,14.083333,0.529395
3,2007-05-04,72.0,50.0,61.25,4.0,41.5,50.0,7.0,0.0,0.0,22.0,14.133333,0.484224
4,2007-05-05,66.0,53.5,60.00,5.0,38.5,49.5,5.0,0.0,0.0,12.5,14.166667,0.448141


In [35]:
# merge test with weather
test = test.merge(weather, on='date', how='left')
test.head()

,id,date,species,latitude,longitude,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,preciptotal,temp_fluct,daylight,relative_hum
0,1,2008-06-11,PIPIENS/RESTUANS,41.95469,-87.800991,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,0.0,22.5,15.166667,0.507584
1,2,2008-06-11,RESTUANS,41.95469,-87.800991,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,0.0,22.5,15.166667,0.507584
2,3,2008-06-11,PIPIENS,41.95469,-87.800991,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,0.0,22.5,15.166667,0.507584
3,4,2008-06-11,SALINARIUS,41.95469,-87.800991,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,0.0,22.5,15.166667,0.507584
4,5,2008-06-11,TERRITANS,41.95469,-87.800991,86.0,63.5,75.0,7.0,55.5,64.0,0.0,10.0,0.0,22.5,15.166667,0.507584


In [36]:
# transform feature matrix
X = pd.DataFrame(ct.fit_transform(test.drop(columns=['id','date'])), 
                       columns=ct.get_feature_names_out())

In [41]:
X.columns

Index(['latitude', 'longitude', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'preciptotal', 'temp_fluct', 'daylight',
       'relative_hum', 'species_ERRATICUS', 'species_IFIED CULEX',
       'species_PIPIENS', 'species_PIPIENS/RESTUANS', 'species_RESTUANS',
       'species_SALINARIUS', 'species_TARSALIS', 'species_TERRITANS'],
      dtype='object')

In [42]:
X.drop(columns=['species_IFIED CULEX'], inplace=True)

In [ ]:
# get predictions in csv format
def get_predictions(model):
    df = pd.DataFrame([test.id, model.predict_proba(X)[:,1]]).T
    df.rename(columns={'Unnamed 0': 'WnvPresent', 'id': 'Id'}, inplace=True)
    df.Id = df.Id.astype(int)
    return df

get_predictions(rfc).to_csv('../output_data/submit_rfc.csv', index=False)
get_predictions(svc).to_csv('../output_data/submit_svc.csv', index=False)
get_predictions(lr).to_csv('../output_data/submit_lr.csv', index=False)
get_predictions(knn).to_csv('../output_data/submit_knn.csv', index=False)
get_predictions(gbc).to_csv('../output_data/submit_gbc.csv', index=False)
get_predictions(xgb).to_csv('../output_data/submit_xgb.csv', index=False)
get_predictions(et).to_csv('../output_data/submit_et.csv', index=False)